In [1]:
from sqlalchemy import create_engine
import pandas as pd 
import numpy as np
mysql_setting = {
    'host' : "localhost",
    'dbname': "spider",
    'user': "root",
    'password': ""
}

def getDatabaseConn():
    engine = create_engine('mysql://{user}:{password}@{host}:3306/{dbname}'.format(**mysql_setting), echo=False)
    conn = engine.connect()
    return conn
conn = getDatabaseConn()

In [2]:
def salarySplit(line):
    import re
    res = re.match(r'([\d]+)K-([\d]+)K', line)
    if not res:
        res = re.match(r'([\d]+)k-([\d]+)k', line)
    if res:
        salary_low = res[1]
        salary_high = res[2]
        salary_mean = (int(salary_low) + int(salary_high)) / 2
    else:
        res = re.match(r'([\d]+).*', line)
        salary_low = res[1]
        salary_high = res[1]
        salary_mean = res[1]
    return pd.Series([salary_low, salary_high, salary_mean])

sql = '''
    select position_name, position_labels, salary, work_year, education, second_type, city
    from lagou_recruit_day
'''
df_lagou = pd.read_sql(sql, conn)
tmp_lagou = df_lagou['salary'].apply(salarySplit).rename(columns={0:'salary_low', 1:'salary_high', 2:'salary_mean'})
df_lagou = df_lagou.combine_first(tmp_lagou)
df_filter = pd.DataFrame(df_lagou, columns=['salary_high', 'salary_low', 'salary_mean'])

# 互联网各工作年限薪资分析

In [3]:
df_lagou[df_lagou.city=='北京'].pivot_table(index=['second_type','work_year']).sort_values('salary_mean',ascending=False)

,,salary_high,salary_low,salary_mean
second_type,work_year,,,
后端开发,10年以上,66.666667,46.666667,56.666667
项目管理,10年以上,60.000000,40.000000,50.000000
[产品设计|需求分析],5-10年,60.000000,30.000000,45.000000
天使投资/VC/PE,5-10年,53.230769,35.846154,44.538462
前端开发/移动开发,10年以上,53.333333,33.333333,43.333333
管理岗,10年以上,52.222222,33.888889,43.055556
并购,5-10年,50.000000,35.000000,42.500000
法务,10年以上,50.000000,35.000000,42.500000
天使投资/VC/PE,10年以上,50.000000,30.000000,40.000000


In [2]:
sql_51job_position = '''
    select salary, work_year, education, city, industry from 51job_position
'''
df_51job_salary = pd.read_sql(sql_51job_position, conn)

In [3]:
df_51job_salary = df_51job_salary[df_51job_salary.salary != 'NULL']

In [4]:
df_51job_salary_deal = df_51job_salary[True ^ df_51job_salary['salary'].str.contains('天|小时|\+')]

In [6]:
def job51SalaryDeal(line):
    import re
    patten1 = r'([\d]+.?[\d]?)-([\d]+.?[\d]?)千/月'
    patten2 = r'([\d]+.?[\d]?)-([\d]+.?[\d]?)万/月'
    patten3 = r'([\d]+.?[\d]?)-([\d]+.?[\d]?)万/年'
    low = high = mean = 0
    if re.compile(patten1).match(line):
        low = float(re.compile(patten1).match(line).group(1))
        high = float(re.compile(patten1).match(line).group(2))
        mean = int((low + high)) / 2
    elif re.compile(patten2).match(line):
        low = float(re.compile(patten2).match(line).group(1)) * 10
        high = float(re.compile(patten2).match(line).group(2)) * 10
        mean = int((low + high)) / 2
    elif re.compile(patten3).match(line):
        low = float(re.compile(patten3).match(line).group(1)) * 10 / 12
        high = float(re.compile(patten3).match(line).group(2)) * 10 / 12
        mean = int(low + high) / 2
    return pd.Series([round(low, 2), round(high, 2), round(mean, 2)])
df_tmp = df_51job_salary_deal.salary.apply(job51SalaryDeal)
df_tmp.head()

,0,1,2
0,4.5,6.0,5.0
1,8.0,10.0,9.0
2,4.5,6.0,5.0
3,4.5,6.0,5.0
4,8.0,9.9,8.5


In [7]:
df_j5_salary = df_51job_salary_deal.combine_first(df_tmp.rename(columns={0 : 'low', 1 : 'high', 2 : 'mean'}))

In [67]:
df_res = df_j5_salary.groupby('education')['mean'].describe().sort_values(['count'], ascending=False)

# 整个行业各教育程度薪资情况

In [71]:
df_res.apply(lambda x: round(x, 2)).rename(index={'NULL': '其他'})

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
大专,985459.0,6.61,5.39,0.0,4.0,5.0,7.5,1250.0
其他,840616.0,7.04,106.23,0.0,3.5,5.0,7.5,94999.5
本科,387949.0,8.92,11.23,0.0,5.0,7.0,10.0,3750.0
高中,183486.0,6.17,4.28,0.0,3.5,5.0,7.0,200.0
中专,161956.0,5.53,6.13,0.0,3.5,5.0,7.0,2000.0
初中及以下,38349.0,5.98,4.45,0.0,3.5,5.0,7.0,125.0
中技,25986.0,5.06,2.88,0.0,3.5,4.5,6.0,85.0
硕士,13242.0,11.11,10.18,0.0,6.5,9.0,12.5,295.0
博士,1054.0,17.93,20.77,0.0,10.0,14.5,20.5,550.0


In [14]:
df_j5_salary.head()

,city,education,high,industry,low,mean,salary,work_year
0,张家港,本科,6.0,制药/生物工程,4.5,5.0,4.5-6千/月,3-4年
1,张家港,本科,10.0,房地产,8.0,9.0,0.8-1万/月,5-7年
2,张家港,中专,6.0,房地产,4.5,5.0,4.5-6千/月,5-7年
3,张家港,NULL,6.0,原材料和加工,4.5,5.0,4.5-6千/月,无
4,张家港,NULL,9.9,互联网/电子商务,8.0,8.5,8-9.9千/月,无


In [75]:
df_j5_salary = df_j5_salary[(True ^ df_j5_salary.industry.isin(['1000-5000人', '5000-10000人', '500-1000人', '少于50人', '150-500人', '50-150人']))]
# df_j5_salary.pivot_table(index=['industry'], values=['mean'], columns=['education'], fill_value=0).apply(lambda x: round(x,2))
df_j5_salary_res = df_j5_salary.groupby('industry')['mean'].describe()

In [84]:
df_j5_industry_salary = df_j5_salary_res[df_j5_salary_res['count'] > 10].sort_values('mean', ascending=False).apply(lambda x:round(x,2))

In [90]:
pd.DataFrame(df_j5_industry_salary, columns=['mean','std'])

,mean,std
industry,,
银行,9.40,6.38
金融/投资/证券,8.96,17.70
房地产,8.92,8.48
租赁服务,8.61,11.95
信托/担保/拍卖/典当,8.47,5.10
网络游戏,8.29,5.38
中介服务,8.04,6.43
多元化业务集团公司,7.82,11.55
互联网/电子商务,7.79,186.36


In [45]:
sql_zl_position = '''
    select position_name, city, salary_low, salary_high, work_year, education, position_type from zhilian_position
'''
df_zl_position = pd.read_sql(sql_zl_position, conn)

# 智联北京软件销售岗 工作年限-学历 薪资待遇

In [53]:
# df_zl_position[(df_zl_position['city']=='北京') & (df_zl_position.position_name.str.contains('软件销售'))].pivot_table(index=['work_year','education']).apply(lambda x:round(x, 2)).rename(columns={'salary_high':'最高薪资', 'salary_low':'最低薪资'})
df_zl_position[(df_zl_position['city']=='北京') & (df_zl_position.position_name.str.contains('软件销售'))].head(30)

,position_name,city,salary_low,salary_high,work_year,education,position_type
91448,软件销售代表,北京,6001,8000,不限,不限,市场专员/助理
138271,软件销售员,北京,4001,6000,1-3年,本科,销售工程师
138493,腾讯企业办公软件销售代表-老员工均薪10K,北京,8001,10000,不限,大专,销售代表
139121,软件销售经理（政府行业）,北京,10000,20000,5-10年,大专,销售经理
139436,金融软件销售代表,北京,8001,10000,不限,大专,销售经理
143450,软件销售代表（提供住宿+带薪培训）,北京,4000,8000,无经验,大专,销售代表
144338,软件销售代表（周末双休、五险一金）18A,北京,6001,8000,不限,不限,客户代表
144586,互联网广告/软件销售 （16A）,北京,6001,8000,不限,大专,客户代表
144858,CRM系统办公软件销售（16A),北京,6000,10000,不限,不限,大客户销售代表
144891,办公软件销售（钉钉CRM系统）16A,北京,5000,10000,无经验,大专,销售代表


In [70]:
df_zl_position.groupby(['position_type']).describe().salary_high.sort_values('50%', ascending=False)[:100]

,count,mean,std,min,25%,50%,75%,max
position_type,,,,,,,,
CTO/CIO,52.0,44942.288462,28307.452358,0.0,23750.0,40000.0,70000.0,120000.0
行长/副行长,44.0,32545.431818,24278.794548,0.0,20000.0,30000.0,50000.0,99999.0
首席财务官CFO,62.0,30983.854839,22811.718961,0.0,15000.0,30000.0,37500.0,100000.0
银行客户总监,16.0,31000.000000,14787.382009,16000.0,20000.0,30000.0,32500.0,70000.0
IT技术/研发总监,116.0,27724.137931,18100.966204,0.0,15000.0,25000.0,40000.0,80000.0
融资总监,107.0,27579.439252,18301.993593,0.0,15000.0,20000.0,40000.0,100000.0
IT项目总监,59.0,24203.389831,17275.435681,0.0,15000.0,20000.0,30000.0,70000.0
合伙人,197.0,40228.416244,46411.891604,0.0,10000.0,20000.0,50000.0,150000.0
算法工程师,326.0,21608.892638,14260.365237,0.0,10000.0,20000.0,30000.0,80000.0


In [125]:
df_city_salary_res = df_j5_salary.groupby('city')['mean'].describe().apply(lambda x:round(x,2))
pd.DataFrame(df_city_salary_res[df_city_salary_res['count'] > 5], columns=['std','mean','50%']).sort_values('mean',ascending=False).rename(columns={'std':'标准差','mean':'平均值','50%':'中位数'})

,标准差,平均值,中位数
city,,,
温州,780.70,13.96,6.00
北京,9.29,9.70,7.00
上海,5.63,8.71,7.00
杭州,19.15,8.38,7.00
深圳,5.99,8.19,7.00
赤峰,13.93,8.08,5.00
衢州,11.36,8.03,6.00
东营,10.74,7.97,6.00
佛山,8.29,7.97,5.00


In [183]:
city_mix_df = pd.concat([pd.DataFrame(df_j5_salary, columns=['city', 'salary_mean', 'work_year']),pd.DataFrame(df_lagou, columns=['city', 'salary_mean', 'work_year'])])
g_df = city_mix_df[city_mix_df['work_year'] == '1-3年'].groupby('city')

# 90后在国内城市岗位薪资排行

In [184]:
g_df_res = g_df.describe().apply(lambda x:round(x,3))
g_df_res = g_df_res[(g_df_res[('salary_mean','std')]<20) & (g_df_res[('salary_mean','count')]>18)].sort_values(('salary_mean','mean'),ascending=False)
pd.DataFrame(g_df_res, columns=[('salary_mean','std'),('salary_mean','mean'),('salary_mean','50%')]).rename(columns={'salary_mean':'薪资/k','mean':'平均数','std':'标准差','50%':'中位数',})

薪资/k               
        标准差     平均数    中位数
city                      
北京    6.249  12.212  11.00
上海    4.976  10.640   9.00
杭州    5.072  10.383   9.00
深圳    4.640   9.668   8.50
南京    3.892   8.513   7.50
广州    3.631   8.425   7.50
珠海    4.054   8.332   7.50
苏州    3.804   8.314   7.50
厦门    3.528   8.011   7.50
宁波    2.846   7.821   7.50
福州    2.994   7.614   7.50
武汉    3.379   7.534   7.00
成都    3.507   7.472   6.50
无锡    3.448   7.468   7.00
东莞    3.157   7.388   7.00
金华    4.086   7.352   7.00
合肥    3.364   7.305   6.50
天津    3.315   7.240   6.50
贵阳    2.683   7.234   6.50
温州    2.636   7.213   7.00
廊坊    2.530   7.148   7.00
乌鲁木齐  2.668   7.051   6.50
长沙    3.110   7.032   6.50
绵阳    2.199   6.969   6.75
南通    4.000   6.964   6.00
中山    2.840   6.957   6.00
重庆    3.143   6.922   6.00
嘉兴    2.409   6.881   6.25
济南    2.556   6.841   6.50
青岛    2.722   6.808   6.00
郑州    2.838   6.680   6.00
呼和浩特  4.417   6.638   5.75
大连    2.728   6.532   6.00
石家庄   3.072   6.512   6.00
西安    2.927   6.500   6.00
沈阳    2.799   6.475   6.00
南宁    2.543   6.439   6.00
临沂    2.585   6.403   6.00
常州    2.199   6.365   6.00
惠州    2.411   6.280   6.00
泉州    2.084   6.192   6.00
南昌    2.748   6.185   6.00
银川    2.007   6.136   6.00
绍兴    1.629   6.107   6.00
潍坊    1.752   6.000   6.00
哈尔滨   2.231   5.866   5.00
烟台    1.788   5.858   6.00
太原    2.248   5.839   5.50
汕头    2.717   5.717   5.00
扬州    2.345   5.712   5.00
长春    2.315   5.704   5.00
桂林    1.833   5.621   6.00
保定    2.004   5.593   5.00
徐州    1.584   5.586   5.50
唐山    1.531   5.431   6.00

# 拉钩销售 职位-工作年限 薪资水平/K

In [44]:
sale_df = df_lagou[(df_lagou.city=='北京') & (df_lagou.position_name.str.contains('软件销售'))].pivot_table(index=['work_year', 'education']).apply(lambda x: round(x, 2)).rename(columns={'salary_high':'最高薪资', 'salary_low':'最低薪资','salary_mean':'平均薪资'})
sale_df

最高薪资  最低薪资   平均薪资
work_year education                   
1-3年      大专         13.0   7.0  10.00
3-5年      大专         10.0   6.0   8.00
不限        不限          8.0   4.0   6.00
          大专          9.5   6.0   7.75
          本科          8.0   4.0   6.00

In [59]:
# 读取51job
j5_df = pd.read_hdf('/Users/monstar/Desktop/ProjectCode/bigdatarecruit/DataMining/salaryWE.h5')
j5_df[(j5_df.city == '北京') & (j5_df['name'].str.contains('软件销售'))].pivot_table(index=['work_year', 'education']).apply(lambda x: round(x, 2)).rename(columns={'high':'最高薪资', 'low':'最低薪资','mean':'平均薪资'})

最高薪资   最低薪资   平均薪资
work_year education                     
1年        NULL        6.00   4.50   5.00
          中专          8.00   6.00   7.00
          大专         13.57   9.00  11.25
          本科          8.00   5.00   6.50
          高中         13.46   8.67  11.06
2年        NULL       12.00   6.00   9.00
          中专         20.00  15.00  17.50
          大专         18.83  14.12  16.47
          本科         12.67   9.67  11.17
3-4年      NULL       25.00  16.67  20.50
          大专         17.09  12.91  15.00
          本科         12.67   8.00  10.33
5-7年      大专         15.00  10.00  12.50
8-9年      大专         25.00  15.00  20.00
          本科         20.00  15.00  17.50
无         NULL       10.21   6.88   8.54
          中专          5.62   3.50   4.50
          大专          9.60   6.58   8.08
          本科          8.60   6.20   7.30
          硕士         10.00   8.00   9.00
          高中          8.00   5.43   6.64

In [62]:
j5_df[(j5_df.city == '北京') & (j5_df['name'].str.contains('软件销售'))].head(30)

,city,education,high,industry,low,mean,name,salary,work_year
3407061,北京,大专,8.0,计算机服务(系统、数据服务、维修),6.0,7.0,软件销售（试用期无责3700+五险+双休+住宿）,6-8千/月,无
3407287,北京,大专,20.0,互联网/电子商务,15.0,17.5,企业软件销售代表（无责底薪4000+高额提成+社保）,1.5-2万/月,2年
3409298,北京,NULL,10.0,互联网/电子商务,5.0,7.5,软件销售（工作地点在燕郊）,0.5-1万/月,无
3410387,北京,大专,10.0,计算机软件,5.0,7.5,软件销售,0.5-1万/月,1年
3410909,北京,NULL,10.0,通信/电信/网络设备,7.0,8.5,软件销售,0.7-1万/月,无
3412048,北京,大专,8.0,互联网/电子商务,6.0,7.0,软件销售代表（0经验，双休，保底4000，社保）,6-8千/月,无
3413280,北京,中专,0.0,计算机软件,0.0,0.0,软件销售专员,2万以下/年,无
3414306,北京,大专,25.0,计算机软件,15.0,20.0,工业软件销售经理（GYSY）,1.5-2.5万/月,8-9年
3414307,北京,大专,15.0,计算机软件,10.0,12.5,工业软件销售工程师（GYSY）,1-1.5万/月,5-7年
3416697,北京,大专,20.0,计算机软件,15.0,17.5,医疗软件销售经理,1.5-2万/月,无
